In [1]:
#########################################  文件路径   #########################################   

path = '/home/wjunneng/Python/ForecastScore'

submission_s1 = '/data/test_s1/submission_s1.csv'
submission_s1_sample_mean_median = '/data/test_s1/submission_s1_sample_mean_median.csv'
submission_s1_sample_svm = '/data/test_s1/submission_s1_sample_svm.csv'
submission_s1_sample_xgb = '/data/test_s1/submission_s1_sample_xgb.csv'
submission_s1_sample_lgb = '/data/test_s1/submission_s1_sample_lgb.csv'

submission_s1_sample_mean_median1 = '/data/test_s1/submission_s1_sample_mean_median1.csv'

course = '/data/train_s1/course.csv'
student = '/data/train_s1/student.csv'
exam_score = '/data/train_s1/exam_score.csv' 
all_knowledge = '/data/train_s1/all_knowledge.csv'
course1_exams = '/data/train_s1/course1_exams.csv'
course2_exams = '/data/train_s1/course2_exams.csv'
course3_exams = '/data/train_s1/course3_exams.csv'
course4_exams = '/data/train_s1/course3_exams.csv'
course5_exams = '/data/train_s1/course5_exams.csv'
course6_exams = '/data/train_s1/course6_exams.csv'
course7_exams = '/data/train_s1/course7_exams.csv'
course8_exams = '/data/train_s1/course8_exams.csv'

submission_s1_sample = '/data/submission_s1_sample.csv'

print('end!')
#########################################  文件路径   #########################################

end!


In [2]:
#########################################  数据加载   #########################################

import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

def get_train_s1(file_name, tag):
    """
    返回train_s1文件夹下的文件
    :param file_name: 文件名称
    :param tag: 返回的数据类型
    :return:
    """
    if tag == 'np':
        return np.loadtxt(path + file_name, delimiter=',', dtype=np.str)
    elif tag == 'pd':
        return pd.read_csv(path + file_name)
    else:
        return "请检查文件名/需要返回的数据类型"

def get_test_s1(file_name, tag):
    """
    返回test_s1文件夹下的文件
    :param file_name: 文件名称
    :param tag: 返回的数据类型
    :return:
    """
    if tag == 'np':
        return np.loadtxt(path + file_name, delimiter=',', dtype=np.str)
    elif tag == 'pd':
        return pd.read_csv(path + file_name)
    else:
        return "请检查文件名/需要返回的数据类型"

def get_sample(file_name, tag):
    """
    返回范例文件夹下的文件
    :param file_name: 文件名称
    :param tag: 返回的数据类型
    :return:
    """
    if tag == 'np':
        return np.loadtxt(path + file_name, delimiter=',', dtype=np.str)
    elif tag == 'pd':
        return pd.read_csv(path + file_name)
    else:
        return "请检查文件名/需要返回的数据类型"

def label_encoding(df, columns: list = None, **params):
    """
    标签编码
    :param df:
    :param params:
    :return:
    """
    if columns is None:
        columns = df.columns

    le = LabelEncoder()
    for feature in columns:
        df[feature] = le.fit_transform(df[feature])

    return df

def get_course(filename, tag='pd'):
    """
    处理course.csv文件
    :param filename:
    :return:
    """
    df = get_train_s1(filename, tag)

    return df

def get_student(filename, tag='pd'):
    """
    处理student.csv文件
    :param filename:
    :return:
    """
    df = get_train_s1(filename, tag)

    return df

def get_course_exams(filename, tag='pd'):
    """
    处理course1_exams.csv-course8_exams.csv
    :param filename:
    :return:
    """
    df = get_train_s1(filename, tag)

    return df

def get_all_knowledge(filename, tag='pd'):
    """
    处理all_knowledge.csv文件
    :param filename:
    :return:
    """
    df = get_train_s1(filename, tag)

    return df

def get_mean_value(df, **params):
    """
    获取每个学生成绩的平均值
    :param df:
    :param params:
    :return:
    """
    # 仅仅取中值
    # mean = df.groupby('student_id').apply(lambda x: x['score'].mean())

    # 去除0值,去除最小值,最大值后取均值
    mean = df.groupby('student_id').apply(lambda x: list(x['score']))

    result = []
    for i in mean.values:
        if 0 in i:
            i.remove(0)
        i.remove(max(i))
        i.remove(min(i))
        result.append(np.array(i).mean())

    mean = pd.Series(result, index=mean.index)
    mean['0'] = result

    return mean

print('end!')
#########################################  数据加载   #########################################

end!


In [3]:
###################################### 查看有0分记录的人次 ########################################

exam_score_data = get_train_s1(exam_score, 'pd')

result = 0
for i in ['course1', 'course2', 'course3', 'course4', 'course5', 'course6', 'course7', 'course8']:
    exam_score_course = exam_score_data[exam_score_data.course == i]
    for exam_score_course_student in exam_score_course.groupby('student_id'):
        tmp = exam_score_course_student[1].score
        if tmp.values.min() == 0:
            result += 1

print('多少人次出现有0分记录：', result)

###################################### 查看有0分记录的人次 ########################################

多少人次出现有0分记录： 146


In [14]:
student_path = '/data/train_s1/student.csv'
course_path = '/data/train_s1/course.csv'
all_knowledge_path = '/data/train_s1/all_knowledge.csv'
exam_score_path = '/data/train_s1/exam_score.csv' 

def merge_all_knowledge(df, course_type=None):
    """
    合并all_knowledge数据
    :param df:
    :param course_type:
    :return:
    """
    # 获取course_exams
    course_exams = get_course_exams(course_type)
    
    # 获取 知识点信息
    all_knowledge = get_all_knowledge(all_knowledge_path)
    # 获取 course_id对应的知识点信息
    all_knowledge = all_knowledge[all_knowledge.course == course_type.split('/')[-1].split('_')[0]]
    
    # 构造 columns
    columns = course_exams.columns.to_list()
    columns.extend(sorted(list(set(all_knowledge['section'])), key=lambda x:int(x.split(':')[-1])))
    columns.extend(sorted(list(set(all_knowledge['category'])), key=lambda x:int(x.split(':')[-1])))
    columns.extend(sorted(['Y:' + str(i) for i in list(set(all_knowledge['complexity']))],  key=lambda x:int(x.split(':')[-1])))

    # 更换 columns
    course_exams = course_exams.reindex(columns=columns)
    # 填充 0
    course_exams.fillna(0, inplace=True)
    
    for i in range(all_knowledge.shape[0]):
        # 获取 一行数据
        line = all_knowledge.iloc[i]
        # 获取 knowledge_point
        knowledge_point = course_exams[line['knowledge_point']]
        # 获取 section
        course_exams[line['section']] += knowledge_point
        # 获取 category
        course_exams[line['category']] += knowledge_point
        # 获取 complexity
        course_exams['Y:' + str(line['complexity'])] += knowledge_point

    # 提取/合并 K：i 知识点
    exam_score = pd.merge(df, course_exams, how='left', on='exam_id')
    
    return exam_score

def get_exam_score(filename, course_id: str, tag='pd', save=True):
    """
    处理exam_score.csv  course_id 只要一个值
    :param filename:
    :return:
    """
    exam_score = get_train_s1(filename, tag)
    
    # 获取 学生信息
    student = get_student(student_path)
    # 获取 课程信息
    course = get_course(course_path)

    # 提取/合并 性别信息
    exam_score = pd.merge(exam_score, student, how='left', on='student_id')
    # 提取/合并 course_class信息
    exam_score = pd.merge(exam_score, course, how='left', on='course')

    # 获取选中的 course_id
    exam_score = exam_score[exam_score['course'] == course_id]

    # 合并 section/category/complexity
    exam_score = merge_all_knowledge(exam_score, course1_exams)
     
    # 读取特定的course_exams.csv文件
    course_exams = get_course_exams('/data/train_s1/course1_exams.csv')
     
    # 处理exam_id
    exam_score['exam_id'] = exam_score['exam_id'].map(
        lambda x: dict(zip(course_exams['exam_id'], [i for i in range(len(course_exams['exam_id']))]))[x])

    # 取均值
    mean_value = get_mean_value(exam_score)

    # 使用均值来填充空值
    result = None
    for tmp in exam_score.groupby('student_id'):
        tmp[1]['score'].replace(0, mean_value[tmp[0]], inplace=True)
        if result is None:
            result = tmp[1]
        else:
            result = pd.concat([result, tmp[1]], axis=0)

    result.reset_index(drop=True, inplace=True)

    exam_score = result

    # log1p就是log(1+x)，用来对得分进行数据预处理，它的好处是转化后的数据更加服从高斯分布，有利于后续的分类结果。
    # 需要注意，最后需要将预测出的平滑数据还原，而还原过程就是log1p的逆运算expm1。
    exam_score["score"] = np.log1p(exam_score["score"])

    ####################################     是否要转str    #######################################
    # exam_score['student_id'] = exam_score['student_id'].astype(str)
    # # 使用.get_dummies()方法对特征矩阵进行类似“坐标投影”操作。获得在新空间下的特征表达。
    # exam_score['student_id'] = pd.get_dummies(exam_score['student_id']).reset_index(drop=True)

    # exam_score['course'] = exam_score['course'].astype(str)
    # exam_score['gender'] = exam_score['gender'].astype(str)
    # exam_score['course_class'] = exam_score['course_class'].astype(str)
    ####################################     是否要转str    #######################################

    # 删除列值都相同的列
    exam_score = exam_score.ix[:, (exam_score != exam_score.ix[0]).any()]
    return exam_score

exam_score = get_exam_score(filename=exam_score_path, course_id='course1')
print(exam_score.head())

/home/wjunneng/Python/anaconda3/envs/lightgbm/lib/python3.6/site-packages/pandas/core/generic.py:6788: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)
/home/wjunneng/Python/anaconda3/envs/lightgbm/lib/python3.6/site-packages/pandas/core/generic.py:6788: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)
/home/wjunneng/Python/anaconda3/envs/lightgbm/lib/python3.6/site-packages/pandas/core/generic.py:6788: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-doc

KeyboardInterrupt: 